In [136]:
import numpy as nu
import sympy as sy

In [137]:
# VALOR DE X
codigo = [ 2, 1, 8, 4, 0, 2, 5 ]
X = sum (codigo)

# 1) Datos.



In [138]:
# Cables en acero:

## longitud en metros
long_cables = 1.5
## Diametro en pulgadas
diam_cables = 1 / 4
## Area en metros cuadrados 
area_cables = nu.pi * ((diam_cables * 25.4 / 1000) ** 2) / 4

# Pasadores:

## diametro en pulgadas
diam_pasadores = 1 / 2
## area en metros cuadrados
area_pasadores = nu.pi * ((diam_pasadores * 25.4 / 1000) ** 2) / 4

# Longitud de la barra:

L = 0.2 * X 

# Factor de seguridad:

## respecto al esfuerzo de fluencia
FS_fluencia = float(f"1.{X}")
## respecto al esfuerzo ultimo
FS_ultimo = float(f"1.{X + 3}")

# Angulos entre cables BE Y BD, BD Y BC (radianes):

## angulo 1 (BE Y BD)
angulo1 = 50 * nu.pi / 180
## angulo 2 (BD Y BC)
angulo2 = 40 * nu.pi / 180

In [139]:
# Propiedades del acero ASTM A-36:

## Esfuerzos de fluencia en MPa
esf_traccion = 250
esf_corte = 145
## Esfuerzo ultimo en MPa
esf_ult = 400
## Modulo de elasticidad en GPa
E = 200

# 2) Solución inciso 1.

In [140]:
GIE = 3

In [141]:
# Ecuaciones de equilibrio

Ax, Ay, FBC, FBD, FBE, W = sy.symbols('Ax Ay FCB FDB FEB W')

# Sumatoria de fuerzas en x

Fx = Ax - FBC * sy.sin(angulo2) + FBE * sy.sin(angulo1)
ecuacion1_X = sy.Eq(Fx, 0)
ecuacion1_X

Eq(Ax - 0.642787609686539*FCB + 0.766044443118978*FEB, 0)

In [142]:
# Sumatoria de fuerzas en y

Fy = Ay - W * L + FBC * sy.cos(angulo2) + FBE * sy.cos(angulo1) + FBD
ecuacion2_Y = sy.Eq(Fy, 0)
ecuacion2_Y

Eq(Ay + 0.766044443118978*FCB + FDB + 0.642787609686539*FEB - 4.4*W, 0)

In [143]:
# Sumatoria de momentos en A 

Ma = - W * L / 2 + FBC * sy.cos(angulo2) * L + FBE * sy.cos(angulo1) * L + FBD * L
ecuacion3_momentos = sy.Eq(Ma, 0)
ecuacion3_momentos

Eq(3.3705955497235*FCB + 4.4*FDB + 2.82826548262077*FEB - 2.2*W, 0)

In [144]:
# Ecuaciones cinematicas

ecuacion4_FBE = sy.Eq(FBE - FBD * sy.cos(angulo1), 0)
expresion_ecuacion4_FBE = FBD * sy.cos(angulo1)
ecuacion4_FBE

Eq(-0.642787609686539*FDB + FEB, 0)

In [145]:
ecuacion5_FBC = sy.Eq(FBC - FBD * sp.cos(angulo2), 0)
expresion_ecuacion5_FBC = FBD * sp.cos(angulo2)
ecuacion5_FBC

Eq(FCB - 0.766044443118978*FDB, 0)

In [146]:
# Reemplazando la ecuacion 4 y 5 en la 3

## Para hallar FBD:

ecuacion6 = sy.Eq(Ma.subs([(FBC, expresion_ecuacion5_FBC), (FBE, expresion_ecuacion4_FBE)]), 0)
ecuacion6

Eq(8.8*FDB - 2.2*W, 0)

In [147]:
FBD_vs_W = sp.solve(ecuacion6)[0][FBD]
FBD_vs_W

0.25*W

In [148]:
BC = expresion_ecuacion5_FBC.subs(FBD, FBD_vs_W)
BE = expresion_ecuacion4_FBE.subs(FBD, FBD_vs_W)

#Fuerzas en los cables:

F_cables = [BC, FBD_vs_W, BE]
F_cables

[0.191511110779745*W, 0.25*W, 0.160696902421635*W]

In [149]:
Wmax = []
Ft = sy.symbols('Fxx')
for fuerza in F_cables:
    # Esfuerzo normal en cables:
    ecuacion7 = sy.Eq(Ft / area_cables, esf_ult * 10 ** 6 / FS_ultimo)
    W_cables = sy.solve(ecuacion7)[0] / 1000
    
    # Cortante simple en pasadores:
    ecuacion8 = sy.Eq(Ft / area_pasadores, esf_corte * 10 ** 6 / FS_fluencia)
    W_pasadores = sy.solve(ecuacion8)[0] / 1000
    print(W_cables, W_pasadores)
    Wmax.append(W_cables)
    Wmax.append(W_pasadores)

print("Carga W máxima en los cables [kN / m]")
min(Wmax)
print()

10.1341495819499 15.0558574731838
10.1341495819499 15.0558574731838
10.1341495819499 15.0558574731838
Carga W máxima en los cables [kN / m]



In [150]:
# Analisis de cortante doble en los apoyos:

Ay_eq = Fy.subs([(FBC, expresion_ecuacion5_FBC), (FBE, expresion_ecuacion4_FBE), (FBD, FBD_vs_W)])
Ay_vs_W = sy.solve(sy.Eq(Ay_eq, 0))
Ay_vs_W = Ay_vs_W[0][Ay]
Ay_vs_W


3.9*W

In [151]:
Ax_eq = Fx.subs([(FBC, expresion_ecuacion5_FBC), (FBE, expresion_ecuacion4_FBE), (FBD, FBD_vs_W)])
Ax_vs_W = sy.solve(sy.Eq(Ax_eq, 0))
Ax_vs_W = Ax_vs_W[0][Ax]
Ax_vs_W

-2.77555756156289e-17*W

In [152]:
# Fuerza resultante en A:

FA = sp.sqrt(Ay_vs_W ** 2 + Ax_vs_W ** 2)
FA

3.9*sqrt(W**2)

In [155]:
# Fuerza en B:

Bx_cort_doble = expresion_ecuacion4_FBE.subs(FBD, FBD_vs_W) * sy.sin(angulo1) - expresion_ecuacion5_FBC.subs(FBD, FBD_vs_W) * sy.sin(angulo2)
By_cort_doble = expresion_ecuacion4_FBE.subs(FBD, FBD_vs_W) * sy.cos(angulo1) + expresion_ecuacion5_FBC.subs(FBD, FBD_vs_W) * sy.cos(angulo2) + FBD_vs_W
FB = sy.sqrt(Bx_cort_doble ** 2 + By_cort_doble ** 2)
FB

0.5*sqrt(W**2)

Entonces;

In [159]:
F_apoyos = [FA, FB]
Wmax_apoyos = []
for fuerza in F_apoyos:
    esf_cort = sy.Eq(fuerza / area_pasadores, esf_corte * 10 ** 6 / FS_fluencia)
    W_pasadores = sp.solve(esf_cort)[0] / 1000
    Wmax_apoyos.append(W_pasadores)

print("Carga W máxima por pasadores [kN / m]")
# carga máxima donde ninguno de los elementos falla
C_max = min(Wmax)
C_max

Carga W máxima por pasadores [kN / m]


10.1341495819499

# 3) Solución inciso 2.

In [160]:
# valor de FBE [unidades fuerza kN y deformación mm]
Fu_BE = expresion_ecuacion4_FBE.subs([(FBD, FBD_vs_W), (W, C_max)])
def_BE = (Fu_BE * 1000 * long_cables) / (area_cables * E * 10 ** 9)

Fu_BE, def_BE * 1000

(1.62852644649686, 0.385672565811923)

In [162]:
# valor de FBC [unidades fuerza kN y deformación mm]
Fu_BC = expresion_ecuacion5_FBC.subs([(FBD, FBD_vs_W), (W, C_max)])
def_BC = (Fu_BC * 1000 * long_cables) / (area_cables * E * 10 ** 9)

Fu_BC, def_BC * 1000

(1.94080224324732, 0.459626665871386)

In [163]:
# valor de FBD [unidades fuerza kN y deformación mm]
Fu_BD = FBD_vs_W.subs(W, C_max)
def_BD = (Fu_BD * 1000 * long_cables) / (area_cables * E * 10 ** 9)
Fu_BD, def_BD * 1000

(2.53353739548749, 0.599999999999999)

In [164]:
# valor de Ax [unidades fuerza kN y deformación mm]
F_Ax = Fx.subs([(FBE, Fu_BE), (FBC, Fu_BC)])
F_Ax = sy.solve(sy.Eq(F_Ax, 0))[0]
F_Ax

-2.22044604925031e-16

In [165]:
# valor de Ay [unidades fuerza kN y deformación mm]
F_Ay = Fy.subs([(FBE, Fu_BE), (FBC, Fu_BC), (FBD, Fu_BD), (W, C_max)])
F_Ay = sy.solve(sy.Eq(F_Ay, 0))[0]
F_Ay

39.5231833696048